# TODO: Nostra

In [ ]:
from typing import Dict, Set
import asyncio
import collections
import copy
import decimal
import sys

# import IPython.display
import pandas

sys.path.append('..')

import src.constants
import src.db
import src.swap_liquidity

## Load and prepare events

In [ ]:
NOSTRA_ETH_ADDRESSES = [
    '0x04f89253e37ca0ab7190b2e9565808f105585c9cacca6b2fa6145553fa061a41',  # ETH
    '0x0553cea5d1dc0e0157ffcd36a51a0ced717efdadd5ef1b4644352bb45bd35453',  # ETH Collateral
    '0x002f8deaebb9da2cb53771b9e2c6d67265d11a4e745ebd74a726b8859c9337b9',  # ETH Interest Bearing
    '0x040b091cb020d91f4a4b34396946b4d4e2a450dbd9410432ebdbfe10e55ee5e5',  # ETH Debt
    '0x070f8a4fcd75190661ca09a7300b7c93fab93971b67ea712c664d7948a8a54c6',  # ETH Interest Bearing Collateral
]
NOSTRA_USDC_ADDRESSES = [
    '0x05327df4c669cb9be5c1e2cf79e121edef43c1416fac884559cd94fcb7e6e232',  # USDC
    '0x047e794d7c49c49fd2104a724cfa69a92c5a4b50a5753163802617394e973833',  # USDC Collateral
    '0x06af9a313434c0987f5952277f1ac8c61dc4d50b8b009539891ed8aaee5d041d',  # USDC Interest Bearing
    '0x03b6058a9f6029b519bc72b2cc31bcb93ca704d0ab79fec2ae5d43f79ac07f7a',  # USDC Debt
    '0x029959a546dda754dc823a7b8aa65862c5825faeaaf7938741d8ca6bfdc69e4e',  # USDC Interest Bearing Collateral
]
NOSTRA_USDT_ADDRESSES = [
    '0x040375d0720245bc0d123aa35dc1c93d14a78f64456eff75f63757d99a0e6a83',  # USDT
    '0x003cd2066f3c8b4677741b39db13acebba843bbbaa73d657412102ab4fd98601',  # USDT Collateral
    '0x06404c8e886fea27590710bb0e0e8c7a3e7d74afccc60663beb82707495f8609',  # USDT Interest Bearing
    '0x065c6c7119b738247583286021ea05acc6417aa86d391dcdda21843c1fc6e9c6',  # USDT Debt
    '0x055ba2baf189b98c59f6951a584a3a7d7d6ff2c4ef88639794e739557e1876f0',  # USDT Interest Bearing Collateral 
]
NOSTRA_DAI_ADDRESSES = [
    '0x02ea39ba7a05f0c936b7468d8bc8d0e1f2116916064e7e163e7c1044d95bd135',  # DAI
    '0x04403e420521e7a4ca0dc5192af81ca0bb36de343564a9495e11c8d9ba6e9d17',  # DAI Collateral
    '0x00b9b1a4373de5b1458e598df53195ea3204aa926f46198b50b32ed843ce508b',  # DAI Interest Bearing
    '0x0362b4455f5f4cc108a5a1ab1fd2cc6c4f0c70597abb541a99cf2734435ec9cb',  # DAI Debt
    '0x01ac55cabf2b79cf39b17ba0b43540a64205781c4b7850e881014aea6f89be58',  # DAI Interest Bearing Collateral
]
NOSTRA_WBTC_ADDRESSES = [
    '0x07788bc687f203b6451f2a82e842b27f39c7cae697dace12edfb86c9b1c12f3d',  # wBTC
    '0x06b59e2a746e141f90ec8b6e88e695265567ab3bdcf27059b4a15c89b0b7bd53',  # wBTC Collateral
    '0x0061d892cccf43daf73407194da9f0ea6dbece950bb24c50be2356444313a707',  # wBTC Interest Bearing
    '0x075b0d87aca8dee25df35cdc39a82b406168fa23a76fc3f03abbfdc6620bb6d7',  # wBTC Debt
    '0x00687b5d9e591844169bc6ad7d7256c4867a10cee6599625b9d78ea17a7caef9',  # wBTC Interest Bearing Collateral
]

NOSTRA_ADDRESSES = [  # TODO: We can probably ignore these
    '0x04f89253e37ca0ab7190b2e9565808f105585c9cacca6b2fa6145553fa061a41',  # ETH
    '0x05327df4c669cb9be5c1e2cf79e121edef43c1416fac884559cd94fcb7e6e232',  # USDC
    '0x040375d0720245bc0d123aa35dc1c93d14a78f64456eff75f63757d99a0e6a83',  # USDT
    '0x02ea39ba7a05f0c936b7468d8bc8d0e1f2116916064e7e163e7c1044d95bd135',  # DAI
    '0x07788bc687f203b6451f2a82e842b27f39c7cae697dace12edfb86c9b1c12f3d',  # wBTC
]
NOSTRA_INTEREST_BEARING_ADDRESSES = [  # TODO: We can probably ignore these
    '0x002f8deaebb9da2cb53771b9e2c6d67265d11a4e745ebd74a726b8859c9337b9',  # ETH Interest Bearing
    '0x06af9a313434c0987f5952277f1ac8c61dc4d50b8b009539891ed8aaee5d041d',  # USDC Interest Bearing
    '0x06404c8e886fea27590710bb0e0e8c7a3e7d74afccc60663beb82707495f8609',  # USDT Interest Bearing
    '0x00b9b1a4373de5b1458e598df53195ea3204aa926f46198b50b32ed843ce508b',  # DAI Interest Bearing
    '0x0061d892cccf43daf73407194da9f0ea6dbece950bb24c50be2356444313a707',  # wBTC Interest Bearing
]
NOSTRA_COLLATERAL_ADDRESSES = [
    '0x0553cea5d1dc0e0157ffcd36a51a0ced717efdadd5ef1b4644352bb45bd35453',  # ETH Collateral
    '0x047e794d7c49c49fd2104a724cfa69a92c5a4b50a5753163802617394e973833',  # USDC Collateral
    '0x003cd2066f3c8b4677741b39db13acebba843bbbaa73d657412102ab4fd98601',  # USDT Collateral
    '0x04403e420521e7a4ca0dc5192af81ca0bb36de343564a9495e11c8d9ba6e9d17',  # DAI Collateral
    '0x06b59e2a746e141f90ec8b6e88e695265567ab3bdcf27059b4a15c89b0b7bd53',  # wBTC Collateral
]
NOSTRA_INTEREST_BEARING_COLLATERAL_ADDRESSES = [
    '0x070f8a4fcd75190661ca09a7300b7c93fab93971b67ea712c664d7948a8a54c6',  # ETH Interest Bearing Collateral
    '0x029959a546dda754dc823a7b8aa65862c5825faeaaf7938741d8ca6bfdc69e4e',  # USDC Interest Bearing Collateral
    '0x055ba2baf189b98c59f6951a584a3a7d7d6ff2c4ef88639794e739557e1876f0',  # USDT Interest Bearing Collateral 
    '0x01ac55cabf2b79cf39b17ba0b43540a64205781c4b7850e881014aea6f89be58',  # DAI Interest Bearing Collateral
    '0x00687b5d9e591844169bc6ad7d7256c4867a10cee6599625b9d78ea17a7caef9',  # wBTC Interest Bearing Collateral
]
NOSTRA_DEBT_ADDRESSES = [
    '0x040b091cb020d91f4a4b34396946b4d4e2a450dbd9410432ebdbfe10e55ee5e5',  # ETH Debt
    '0x03b6058a9f6029b519bc72b2cc31bcb93ca704d0ab79fec2ae5d43f79ac07f7a',  # USDC Debt
    '0x065c6c7119b738247583286021ea05acc6417aa86d391dcdda21843c1fc6e9c6',  # USDT Debt
    '0x0362b4455f5f4cc108a5a1ab1fd2cc6c4f0c70597abb541a99cf2734435ec9cb',  # DAI Debt
    '0x075b0d87aca8dee25df35cdc39a82b406168fa23a76fc3f03abbfdc6620bb6d7',  # wBTC Debt
]

ALL_RELEVANT_NOSTRA_ADDRESSES = NOSTRA_COLLATERAL_ADDRESSES + NOSTRA_INTEREST_BEARING_COLLATERAL_ADDRESSES + NOSTRA_DEBT_ADDRESSES

NOSTRA_INTEREST_MODEL_UPDATES_ADDRESS = '0x03d39f7248fb2bfb960275746470f7fb470317350ad8656249ec66067559e892'

NOSTRA_DEBT_ADDRESSES_TO_TOKEN = {
    # TODO: remove the first `0`'s after the `x`, e.g. `0x040...` -> `0x40...`
    '0x40b091cb020d91f4a4b34396946b4d4e2a450dbd9410432ebdbfe10e55ee5e5': 'ETH',  # ETH Debt
    '0x3b6058a9f6029b519bc72b2cc31bcb93ca704d0ab79fec2ae5d43f79ac07f7a': 'USDC',  # USDC Debt
    '0x65c6c7119b738247583286021ea05acc6417aa86d391dcdda21843c1fc6e9c6': 'USDT',  # USDT Debt
    '0x362b4455f5f4cc108a5a1ab1fd2cc6c4f0c70597abb541a99cf2734435ec9cb': 'DAI',  # DAI Debt
    '0x75b0d87aca8dee25df35cdc39a82b406168fa23a76fc3f03abbfdc6620bb6d7': 'wBTC',  # wBTC Debt
}

In [ ]:
# Establish the connection.
connection = src.db.establish_connection()

# Load all Nostra events.
nostra_events = pandas.read_sql(
    sql = f"""
        SELECT
            *
        FROM
            starkscan_events
        WHERE
            (
                from_address IN {tuple(ALL_RELEVANT_NOSTRA_ADDRESSES)}
            AND
                key_name IN ('Burn', 'Mint')
            )
        OR 
            (
                from_address = '{NOSTRA_INTEREST_MODEL_UPDATES_ADDRESS}'
            AND
                key_name = 'InterestStateUpdated'
            )
        ORDER BY
            block_number, id ASC;
    """,
    con = connection,
)

# Close the connection.
connection.close()

In [ ]:
nostra_events.set_index('id', inplace = True)
nostra_events

In [ ]:
# TODO
print({x: len(nostra_events[nostra_events['key_name'] == x]) for x in nostra_events['key_name'].unique()})

# Process events

Events examples:
- [collateral:Mint](https://starkscan.co/event/0x015dccf7bc9a434bcc678cf730fa92641a2f6bcbfdb61cbe7a1ef7d0a614d1ac_3)
- [collateral:Burn](https://starkscan.co/event/0x00744177ee88dd3d96dda1784e2dff50f0c989b7fd48755bc42972af2e951dd6_1)
- [interest bearing collateral:Mint](https://starkscan.co/event/0x07d222d9a70edbe717001ab4305a7a8cfb05116a35da24a9406209dbb07b6d0b_5)
- [interest bearing collateral:Burn](https://starkscan.co/event/0x0106494005bbab6f01e7779760891eb9ae20e01b905afdb16111f7cf3a28a53e_1)
- [debt:Mint](https://starkscan.co/event/0x030d23c4769917bc673875e107ebdea31711e2bdc45e658125dbc2e988945f69_4)
- [debt:Burn](https://starkscan.co/event/0x002e4ee376785f687f32715d8bbed787b6d0fa9775dc9329ca2185155a139ca3_5)

- [InterestStateUpdated](https://starkscan.co/event/0x05e95588e281d7cab6f89aa266057c4c9bcadf3ff0bb85d4feea40a4faa94b09_4)

In [ ]:
# TODO: create a proper mapping
def get_token(address: str) -> str:
    if address in NOSTRA_ETH_ADDRESSES:
        return 'ETH'
    if address in NOSTRA_USDC_ADDRESSES:
        return 'USDC'
    if address in NOSTRA_USDT_ADDRESSES:
        return 'USDT'
    if address in NOSTRA_DAI_ADDRESSES:
        return 'DAI'
    if address in NOSTRA_WBTC_ADDRESSES:
        return 'wBTC'


class InterestModelState:
    """
    TODO
    """

    def __init__(self) -> None:
        self.lend_index: decimal.Decimal = decimal.Decimal("1e18")  # Reflects interest rate at which users lend.
        self.borrow_index: decimal.Decimal = decimal.Decimal("1e18")  # Reflects interest rate at which users borrow.

    def interest_model_update(self, lend_index: decimal.Decimal, borrow_index: decimal.Decimal):
        self.lend_index = lend_index / decimal.Decimal("1e18")
        self.borrow_index = borrow_index / decimal.Decimal("1e18")


class UserTokenState:
    """
    TODO
    """

    MAX_ROUNDING_ERRORS = {
        "ETH": decimal.Decimal("0.5") * decimal.Decimal("1e13"),
        "wBTC": decimal.Decimal("1e2"),
        "USDC": decimal.Decimal("1e4"),
        "DAI": decimal.Decimal("1e16"),
        "USDT": decimal.Decimal("1e4"),
        "wstETH": decimal.Decimal("0.5") * decimal.Decimal("1e13"),
    }

    def __init__(self, token: str) -> None:
        self.token: str = token
        self.collateral: decimal.Decimal = decimal.Decimal("0")
        self.interest_bearing_collateral: decimal.Decimal = decimal.Decimal("0")
        self.debt: decimal.Decimal = decimal.Decimal("0")

    def update_collateral(self, raw_amount: decimal.Decimal):
        self.collateral += raw_amount
        if (
            -self.MAX_ROUNDING_ERRORS[self.token]
            < self.collateral
            < self.MAX_ROUNDING_ERRORS[self.token]
        ):
            self.collateral = decimal.Decimal("0")

    def update_interest_bearing_collateral(self, raw_amount: decimal.Decimal):
        self.interest_bearing_collateral += raw_amount
        if (
            -self.MAX_ROUNDING_ERRORS[self.token]
            < self.interest_bearing_collateral
            < self.MAX_ROUNDING_ERRORS[self.token]
        ):
            self.interest_bearing_collateral = decimal.Decimal("0")

    def update_debt(self, raw_amount: decimal.Decimal):
        self.debt += raw_amount
        if (
            -self.MAX_ROUNDING_ERRORS[self.token]
            < self.debt
            < self.MAX_ROUNDING_ERRORS[self.token]
        ):
            self.debt = decimal.Decimal("0")


class UserState:
    """
    TODO
    """

    def __init__(self) -> None:
        self.token_states: Dict[str, UserTokenState] = {
            "ETH": UserTokenState("ETH"),
            "wBTC": UserTokenState("wBTC"),
            "USDC": UserTokenState("USDC"),
            "DAI": UserTokenState("DAI"),
            "USDT": UserTokenState("USDT"),
        }


class State:
    """
    TODO
    """

    USER = "0x5fe47a2abea8664fbbdd3c0ec52cdf027019eb5162ae015cdaecad4108cab34"

    def __init__(self) -> None:
        self.user_states: collections.defaultdict = collections.defaultdict(UserState)
        self.interest_model_states: Dict[str, InterestModelState] = {
            "ETH": InterestModelState(),
            "wBTC": InterestModelState(),
            "USDC": InterestModelState(),
            "DAI": InterestModelState(),
            "USDT": InterestModelState(),
        }

    def process_event(self, event: pandas.Series) -> None:
        if event['from_address'] == NOSTRA_INTEREST_MODEL_UPDATES_ADDRESS:
            self.process_interest_model_update_event(event)

        is_collateral = event['from_address'] in NOSTRA_COLLATERAL_ADDRESSES
        is_interest_bearing_collateral = event['from_address'] in NOSTRA_INTEREST_BEARING_COLLATERAL_ADDRESSES
        is_debt = event['from_address'] in NOSTRA_DEBT_ADDRESSES

        # TODO: do this in a better way?
        if is_collateral:
            self.process_collateral_event(event)
        if is_interest_bearing_collateral:
            self.process_interest_bearing_collateral_event(event)
        if is_debt:
            self.process_debt_event(event)

    def process_interest_model_update_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `debtToken`, `lendingRate`, `borrowRate`, `lendIndex`, `borrowIndex`.
        token = NOSTRA_DEBT_ADDRESSES_TO_TOKEN[event["data"][0]]
        lend_index = decimal.Decimal(str(int(event["data"][5], base=16)))
        borrow_index = decimal.Decimal(str(int(event["data"][7], base=16)))
        self.interest_model_states[token].interest_model_update(
            lend_index=lend_index,
            borrow_index=borrow_index,
        )

    def process_collateral_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `user`, `amount`.
        name = event["key_name"]
        user = event['data'][0]
        # TODO: This seems to be a magical address. Let's first find out what its purpose is.
        if user == '0x5fc7053cca20fcb38550d7554c84fa6870e2b9e7ebd66398a67697ba440f12b':
            return
        token = get_token(event['from_address'])
        amount = decimal.Decimal(str(int(event['data'][1], base=16)))
        raw_amount = amount / self.interest_model_states[token].lend_index
        if name == 'Mint':  # Collateral deposited.
            self.user_states[user].token_states[token].update_collateral(raw_amount = raw_amount)
        if name == 'Burn':  # Collateral withdrawn.
            self.user_states[user].token_states[token].update_collateral(raw_amount = -raw_amount)
        # TODO
        if user == self.USER:
            print(event['block_number'], "col", name, token, amount)

    def process_interest_bearing_collateral_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `user`, `amount`.
        name = event["key_name"]
        user = event['data'][0]
        # TODO: This seems to be a magical address. Let's first find out what its purpose is.
        if user == '0x5fc7053cca20fcb38550d7554c84fa6870e2b9e7ebd66398a67697ba440f12b':
            return
        token = get_token(event['from_address'])
        amount = decimal.Decimal(str(int(event['data'][1], base=16)))
        raw_amount = amount / self.interest_model_states[token].lend_index
        if name == 'Mint':  # Collateral deposited.
            self.user_states[user].token_states[token].update_interest_bearing_collateral(raw_amount = raw_amount)
        if name == 'Burn':  # Collateral withdrawn.
            self.user_states[user].token_states[token].update_interest_bearing_collateral(raw_amount = -raw_amount)
        # TODO
        if user == self.USER:
            print(event['block_number'], "ib col", name, token, amount)

    def process_debt_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `user`, `amount`.
        name = event["key_name"]
        user = event['data'][0]
        token = get_token(event['from_address'])
        amount = decimal.Decimal(str(int(event['data'][1], base=16)))
        raw_amount = amount / self.interest_model_states[token].borrow_index
        if name == 'Mint':  # Debt borrowed.
            self.user_states[user].token_states[token].update_debt(raw_amount = raw_amount)
        if name == 'Burn':  # Debt repayed.
            self.user_states[user].token_states[token].update_debt(raw_amount = -raw_amount)
        # TODO
        if user == self.USER:
            print(event['block_number'], "deb", name, token, amount)

In [ ]:
# TODO
# Iterate over ordered events to obtain the final state of each user.
state = State()
for _, event in nostra_events.iterrows():
    state.process_event(event = event)

In [ ]:
# TODO: just a simple sanity check
prices = src.swap_liquidity.Prices()
for user, user_state in state.user_states.items():
    for token, token_state in user_state.token_states.items():
        if token_state.collateral < 0:
            collateral = token_state.collateral / src.constants.TOKEN_DECIMAL_FACTORS[token] * prices.prices[token]
            if collateral < -2:
                print('negative collateral', user, token, collateral)
        if token_state.interest_bearing_collateral < 0:
            interest_bearing_collateral = token_state.interest_bearing_collateral / src.constants.TOKEN_DECIMAL_FACTORS[token] * prices.prices[token]
            if interest_bearing_collateral < -2:
                print('negative interest bearing collateral', user, token, interest_bearing_collateral)
        if token_state.debt < 0:
            debt = token_state.debt / src.constants.TOKEN_DECIMAL_FACTORS[token] * prices.prices[token]
            if debt < -2:
                print('negative debt', user, token, debt)

In [ ]:
# TODO
for token, token_state in state.user_states['0x5fe47a2abea8664fbbdd3c0ec52cdf027019eb5162ae015cdaecad4108cab34'].token_states.items():
    print(token, token_state.collateral, token_state.interest_bearing_collateral, token_state.debt)

In [ ]:
# TODO
nostra_events[nostra_events['block_number'] == 135939]

# Compute max liquidated borrowings

TODO: This part is new. Will be refactored.

In [ ]:
# Get current prices of tokens.
prices = src.swap_liquidity.Prices()

In [ ]:
# TODO: find collateral factors
COLLATERAL_FACTORS = {
    'ETH': decimal.Decimal('0.8'),  # https://starkscan.co/call/0x06f619127a63ddb5328807e535e56baa1e244c8923a3b50c123d41dcbed315da_1_1
    'USDC': decimal.Decimal('0.9'),  # https://starkscan.co/call/0x0540d0c76da67ed0e4d6c466c75f14f9b34a7db743b546a3556125a8dbd4b013_1_1
    'USDT': decimal.Decimal('0.9'),  # https://starkscan.co/call/0x020feb60fb3360e9dcbe9ddb8157334d3b95c0758c9df141d6398c72ffd4aa56_1_1
    # TODO: verify via chain call?
    'DAI': decimal.Decimal('0'),  # https://starkscan.co/call/0x057c4cdc434e83d68f4fd004d9cd34cb73f4cc2ca6721b88da488cfbf2d33ec9_1_1
    # TODO: verify via chain call?
    'wBTC': decimal.Decimal('0'),  # https://starkscan.co/call/0x06d02f87f6d5673ea414bebb58dbe24bfbc7abd385b45710b12562f79f0b602c_1_1
}
DEBT_FACTORS = {
    'ETH': decimal.Decimal('0.9'),
    'USDC': decimal.Decimal('0.95'),
    'USDT': decimal.Decimal('0.95'),
    'DAI': decimal.Decimal('0.95'),
    'wBTC': decimal.Decimal('0.8'),
}

LIQUIDATION_HEALTH_FACTOR_THRESHOLD = decimal.Decimal('1')
TARGET_HEALTH_FACTOR = decimal.Decimal('1.25')  # TODO
LIQUIDATOR_FEE_BETAS = {
    'ETH': decimal.Decimal('2.75'),
    'USDC': decimal.Decimal('1.65'),
    'USDT': decimal.Decimal('1.65'),
    'DAI': decimal.Decimal('2.2'),
    'wBTC': decimal.Decimal('2.75'),
}
LIQUIDATOR_FEE_MAXS = {
    'ETH': decimal.Decimal('0.25'),
    'USDC': decimal.Decimal('0.15'),
    'USDT': decimal.Decimal('0.15'),
    'DAI': decimal.Decimal('0.2'),
    'wBTC': decimal.Decimal('0.25'),
}
PROTOCOL_FEES = {
    'ETH': decimal.Decimal('0.02'),
    'USDC': decimal.Decimal('0.02'),
    'USDT': decimal.Decimal('0.02'),
    'DAI': decimal.Decimal('0.02'),
    'wBTC': decimal.Decimal('0.02'),
}

# Source: Starkscan, e.g.
# https://starkscan.co/token/0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7 for ETH.
TOKEN_DECIMAL_FACTORS = {
    "ETH": decimal.Decimal('1e18'),
    "wBTC": decimal.Decimal('1e8'),
    "USDC": decimal.Decimal('1e6'),
    "DAI": decimal.Decimal('1e18'),
    "USDT": decimal.Decimal('1e6'),
}


def compute_risk_adjusted_collateral_usd(
    user_state: UserState,
    prices: Dict[str, decimal.Decimal],
) -> decimal.Decimal:
    return sum(
        (token_state.collateral + token_state.interest_bearing_collateral)
        * COLLATERAL_FACTORS[token]
        * prices[token]
        # TODO: perform the conversion using TOKEN_DECIMAL_FACTORS sooner (in `UserTokenState`?)?
        / TOKEN_DECIMAL_FACTORS[token]
        for token, token_state in user_state.token_states.items()
    )


def compute_risk_adjusted_debt_usd(
    user_state: UserState,
    prices: Dict[str, decimal.Decimal],
) -> decimal.Decimal:
    return sum(
        token_state.debt
        / DEBT_FACTORS[token]
        * prices[token]
        # TODO: perform the conversion using TOKEN_DECIMAL_FACTORS sooner (in `UserTokenState`?)?
        / TOKEN_DECIMAL_FACTORS[token]
        for token, token_state in user_state.token_states.items()
    )


def compute_health_factor(
    risk_adjusted_collateral_usd: decimal.Decimal,
    risk_adjusted_debt_usd: decimal.Decimal,
) -> decimal.Decimal:
    if risk_adjusted_debt_usd == decimal.Decimal("0"):
        # TODO: assumes collateral is positive
        return decimal.Decimal("Inf")

    health_factor = risk_adjusted_collateral_usd / risk_adjusted_debt_usd
    # TODO: enable?
    #     if health_factor < decimal.Decimal('0.9'):
    #         print(f'Suspiciously low health factor = {health_factor} of user = {user}, investigate.')
    # TODO: too many loans eligible for liquidation?
    #     elif health_factor < decimal.Decimal('1'):
    #         print(f'Health factor = {health_factor} of user = {user} eligible for liquidation.')
    return health_factor


# TODO: compute_health_factor, etc. should be methods of class UserState
def compute_debt_to_be_liquidated(
    debt_token: str,
    collateral_tokens: Set[str],
    health_factor: decimal.Decimal,
    debt_token_debt_amount: decimal.Decimal,
    debt_token_price: decimal.Decimal,
) -> decimal.Decimal:
    liquidator_fee_usd = decimal.Decimal('0')
    liquidation_amount_usd = decimal.Decimal('0')
    for collateral_token in collateral_tokens:
        # TODO: commit the derivation of the formula in a document?
        # See an example of a liquidation here: https://docs.nostra.finance/lend/liquidations/an-example-of-liquidation.
        liquidator_fee = min(
            LIQUIDATOR_FEE_BETAS[collateral_token] * (LIQUIDATION_HEALTH_FACTOR_THRESHOLD - health_factor),
            LIQUIDATOR_FEE_MAXS[collateral_token],
        )
        total_fee = liquidator_fee + PROTOCOL_FEES[collateral_token]
        max_liquidation_percentage = (
            TARGET_HEALTH_FACTOR - health_factor
        ) / (
            TARGET_HEALTH_FACTOR - COLLATERAL_FACTORS[collateral_token] * DEBT_FACTORS[debt_token] * (decimal.Decimal('1') + total_fee)
        )
        max_liquidation_percentage = min(max_liquidation_percentage, decimal.Decimal('1'))
        max_liquidation_amount = max_liquidation_percentage * debt_token_debt_amount
        max_liquidation_amount_usd = max_liquidation_amount * debt_token_price / TOKEN_DECIMAL_FACTORS[debt_token]
        max_liquidator_fee_usd = liquidator_fee * max_liquidation_amount_usd
        if max_liquidator_fee_usd > liquidator_fee_usd:
            liquidator_fee_usd = max_liquidator_fee_usd
            liquidation_amount_usd = max_liquidation_amount_usd
    return liquidation_amount_usd


def compute_max_liquidated_amount(
    state: State,
    prices: Dict[str, decimal.Decimal],
    debt_token: str,
) -> decimal.Decimal:
    liquidated_debt_amount = decimal.Decimal("0")
    for user, user_state in state.user_states.items():
        # TODO: do this?
        # Filter out users who borrowed the token of interest.
        debt_tokens = {
            token_state.token
            for token_state in user_state.token_states.values()
            if token_state.debt > decimal.Decimal("0")
        }
        if not debt_token in debt_tokens:
            continue

        # Filter out users with health below 1.
        risk_adjusted_collateral_usd = compute_risk_adjusted_collateral_usd(
            user_state=user_state,
            prices=prices,
        )
        risk_adjusted_debt_usd = compute_risk_adjusted_debt_usd(
            user_state=user_state,
            prices=prices,
        )
        health_factor = compute_health_factor(
            risk_adjusted_collateral_usd=risk_adjusted_collateral_usd,
            risk_adjusted_debt_usd=risk_adjusted_debt_usd,
        )
        if health_factor >= decimal.Decimal('1'):
            continue

        # TODO: find out how much of the debt_token will be liquidated
        collateral_tokens = {
            token_state.token
            for token_state in user_state.token_states.values()
            if token_state.collateral
            != decimal.Decimal("0")
            or token_state.interest_bearing_collateral
            != decimal.Decimal("0")
        }
        liquidated_debt_amount += compute_debt_to_be_liquidated(
            debt_token=debt_token,
            collateral_tokens=collateral_tokens,
            health_factor=health_factor,
            debt_token_debt_amount=user_state.token_states[debt_token].debt,
            debt_token_price=prices[debt_token],
        )
    return liquidated_debt_amount


# TODO: this function is general for all protocols
def decimal_range(start: decimal.Decimal, stop: decimal.Decimal, step: decimal.Decimal):
    while start < stop:
        yield start
        start += step


def simulate_liquidations_under_absolute_price_change(
    prices: src.swap_liquidity.Prices,
    collateral_token: str,
    collateral_token_price: decimal.Decimal,
    state: State,
    debt_token: str,
) -> decimal.Decimal:
    changed_prices = copy.deepcopy(prices.prices)
    changed_prices[collateral_token] = collateral_token_price
    return compute_max_liquidated_amount(
        state=state, prices=changed_prices, debt_token=debt_token
    )


# TODO: this function is general for all protocols
def get_amm_supply_at_price(
    collateral_token: str,
    collateral_token_price: decimal.Decimal,
    debt_token: str,
    amm: src.swap_liquidity.SwapAmm,
) -> decimal.Decimal:
    return amm.get_pool(collateral_token, debt_token).supply_at_price(
        debt_token, collateral_token_price
    )

In [ ]:
# # TODO
# for user, user_state in state.user_states.items():
#     risk_adjusted_collateral_usd = compute_risk_adjusted_collateral_usd(
#         user_state=user_state,
#         prices=prices.prices,
#     )
#     risk_adjusted_debt_usd = compute_risk_adjusted_debt_usd(
#         user_state=user_state,
#         prices=prices.prices,
#     )
#     health_factor = compute_health_factor(
#         risk_adjusted_collateral_usd=risk_adjusted_collateral_usd,
#         risk_adjusted_debt_usd=risk_adjusted_debt_usd,
#     )
#     if health_factor == 0:
#         print(health_factor, risk_adjusted_collateral_usd, risk_adjusted_debt_usd)

In [ ]:
# TODO
COLLATERAL_TOKEN = 'ETH'
DEBT_TOKEN = 'USDC'


data = pandas.DataFrame(
    {
        "collateral_token_price": [
            x
            for x in decimal_range(
                # TODO: make it dependent on the collateral token .. use prices.prices[COLLATERAL_TOKEN]
                start=decimal.Decimal("1000"),
                stop=decimal.Decimal("3000"),
                # TODO: make it dependent on the collateral token
                step=decimal.Decimal("50"),
            )
        ]
    },
)
data["max_debt_to_be_liquidated"] = data["collateral_token_price"].apply(
    lambda x: simulate_liquidations_under_absolute_price_change(
        prices=prices,
        collateral_token=COLLATERAL_TOKEN,
        collateral_token_price=x,
        state=state,
        debt_token=DEBT_TOKEN,
    )
)

# TODO
data["max_debt_to_be_liquidated_at_interval"] = (
    data["max_debt_to_be_liquidated"].diff().abs()
)
# TODO: drops also other NaN, if there are any
data.dropna(inplace=True)

# Setup the AMMs.
swap_amms = await src.swap_liquidity.SwapAmm().init()

data["amm_debt_token_supply"] = data["collateral_token_price"].apply(
    lambda x: get_amm_supply_at_price(
        collateral_token=COLLATERAL_TOKEN,
        collateral_token_price=x,
        debt_token=DEBT_TOKEN,
        amm=swap_amms,
    )
)

In [ ]:
# TODO
data

In [ ]:
# # TODO
# import plotly.express


# figure = plotly.express.histogram(
#     loan_stats.loc[loan_stats['Health factor'] < 5],#.astype(float),
#     x = 'Health factor',
#     nbins = 100,
#     title = 'Health factor histogram',
# )
# figure.show()

In [ ]:
# # TODO: check numbers of borrowings-collateral token pairs
# import itertools


# counts = {}
# for b, c in itertools.product(['wBTC', 'ETH', 'USDC', 'DAI', 'USDT'], ['wBTC', 'ETH', 'USDC', 'DAI', 'USDT']):
#     count = 0
#     for user, user_state in state.user_states.items():
#         for loan_id, loan in user_state.loans.items():
#             if loan.borrowings.market == b and loan.collateral.market == c:
#                 count += 1
#     counts[f'{b} - {c}'] = count
# counts

## Plot the liquidated and supply amounts

In [ ]:
import plotly.express


figure = plotly.express.bar(
    data.astype(float),
    x = 'collateral_token_price',
    y = ['max_debt_to_be_liquidated', 'amm_debt_token_supply'],
    title = f'Potentially liquidatable amounts of {DEBT_TOKEN} and the corresponding supply',
    barmode = 'overlay',
    opacity = 0.65,
)
figure.show()